In [109]:
import numpy as np
import cv2

def preprocess_mnist_style(img_path):
    # 1️⃣ Load grayscale
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Image not found: {img_path}")

    # 2️⃣ Threshold to get binary image (black/white)
    _, img_bin = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV)  # digit white, background black

    # 3️⃣ Find bounding box of the digit
    contours, _ = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        raise ValueError("No digit found in the image!")
    x, y, w, h = cv2.boundingRect(contours[0])
    digit = img_bin[y:y+h, x:x+w]

    # 4️⃣ Resize while keeping aspect ratio
    # Target size: 20x20 (like MNIST)
    h_new = 20
    w_new = int(w * (20 / h)) if h > w else 20
    digit_resized = cv2.resize(digit, (w_new, h_new))

    # 5️⃣ Place in 28x28 canvas, centered
    canvas = np.zeros((28,28), dtype=np.uint8)
    x_offset = (28 - w_new) // 2
    y_offset = (28 - h_new) // 2
    canvas[y_offset:y_offset+h_new, x_offset:x_offset+w_new] = digit_resized

    # 6️⃣ Normalize to [0,1]
    canvas = canvas.astype("float32") / 255.0

    # 7️⃣ Flatten for DNN
    canvas = canvas.reshape(1, 784)

    return canvas


This function takes an image of a handwritten digit and converts it into a MNIST-like format suitable for a DNN trained on MNIST digits.

Loads the image as grayscale (single channel).

Inversion (THRESH_BINARY_INV) ensures the digit matches MNIST format.

canvas = np.zeros((28,28), dtype=np.uint8)
Creates a 28x28 black canvas (MNIST size).

Flattened DNNs are very sensitive to position and scale.

This preprocessing makes handwritten digit look like MNIST, so the model can recognize it better.

(The  handwritten image which is passed is taken from camscanner and  it is scanned and then it is preprocessed in code)

In [111]:
X = preprocess_mnist_style("/content/result_num (3).jpg")
pred = model.predict(X)
label = np.argmax(pred)
print("Predicted Digit:", label)
print("Confidence:", np.max(pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Predicted Digit: 8
Confidence: 0.9994654


In [112]:
import numpy as np
import cv2
from keras.models import load_model

# 1️⃣ Load the DNN model
model_path = "/content/mnist_dnn.keras"
model = load_model(model_path)
print("✅ Model loaded successfully!")

# 2️⃣ MNIST-style preprocessing
def preprocess_mnist_style(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Image not found: {img_path}")

    # Threshold & invert: digit white, background black
    _, img_bin = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV)

    # Find bounding box of digit
    contours, _ = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        raise ValueError("No digit found in the image!")
    x, y, w, h = cv2.boundingRect(contours[0])
    digit = img_bin[y:y+h, x:x+w]

    # Resize keeping aspect ratio
    h_new = 20
    w_new = int(w * (20 / h)) if h > w else 20
    digit_resized = cv2.resize(digit, (w_new, h_new))

    # Center in 28x28 canvas
    canvas = np.zeros((28,28), dtype=np.uint8)
    x_offset = (28 - w_new) // 2
    y_offset = (28 - h_new) // 2
    canvas[y_offset:y_offset+h_new, x_offset:x_offset+w_new] = digit_resized

    # Normalize & flatten
    canvas = canvas.astype("float32") / 255.0
    canvas = canvas.reshape(1, 784)

    return canvas

# Preprocess your image
X = preprocess_mnist_style("/content/result_num (3).jpg")
print("Input shape:", X.shape)

# 3️⃣ Predict digit
pred = model.predict(X)
label = np.argmax(pred)
print("\n🎯 Predicted Digit:", label)
print("Confidence:", np.max(pred))

# 4️⃣ Layer-by-layer outputs
intermediate_outputs = []
input_to_layer = X

for i, layer in enumerate(model.layers):
    output = layer(input_to_layer)           # call each layer manually
    intermediate_outputs.append(output.numpy())
    input_to_layer = output

# 5️⃣ Print outputs
print("\n🔍 LAYER-BY-LAYER OUTPUTS:")
for i, out in enumerate(intermediate_outputs):
    print(f"\n----- Layer {i}: {model.layers[i].name} -----")
    print("Shape:", out.shape)
    print(out)


✅ Model loaded successfully!
Input shape: (1, 784)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step

🎯 Predicted Digit: 8
Confidence: 0.9994654

🔍 LAYER-BY-LAYER OUTPUTS:

----- Layer 0: dense_3 -----
Shape: (1, 256)
[[0.         1.9673034  0.         0.         0.         0.
  4.397518   0.         0.         0.         0.         0.
  0.         0.         1.5611564  0.         0.         0.
  0.         1.7604582  0.20794944 0.         0.         0.
  0.         0.6956392  0.78662807 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         1.480578   0.         0.         0.         1.8665639
  0.         0.         0.23447591 0.         0.6605591  0.
  0.         0.         0.         0.         0.71121395 0.
  0.         0.         0.         0.         0.         2.0043712
  0.         0.         0.         0.         0.         2.916184
  0.20158628 0.         1.8172084  0.         0.         1.9014429
  0.         0.         0.         1.2752706  

Loads the pre-trained MNIST DNN model (mnist_dnn.keras).

This model was trained on flattened 28×28 MNIST digits (784 input features).

Prediction depends entirely on how close your input resembles MNIST digits.

_, img_bin = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV) : Converts to binary (0 or 255).
Inverts so digit = white, background = black.
(Focuses only on the digit, removes empty background.)

Converts pixel values to [0,1] (DNN expects normalized input).

np.argmax(pred) selects the digit with highest probability.

Example:

Predicted Digit: 8

Confidence: 0.98

Why the model predicted this value:

Your input digit looked most similar to 8 in the model’s learned feature space.

Flattened DNNs are sensitive to input style.

Preprocessing mismatch

If the digit wasn’t perfectly centered or inverted, some features might resemble another number.

MNIST-trained DNN limitations

 handwritten digit looks different at the pixel level than the MNIST digits the model saw during training, so the DNN predicts the closest-looking digit in its learned pixel patterns.
   

   

